<a href="https://colab.research.google.com/github/BeforeTheStorm202/DACN/blob/main/InSDN_hsj_CNN_SHAP_detection_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 10.4 MB/s eta 0:00:00


In [2]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Load trainset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [4]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

# Load adv samples

In [5]:
hsj_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/24_feature/hsj/hsj_cnn_bin_feature_10000'

In [6]:
hsj_df = pd.read_csv(hsj_path)

In [7]:
hsj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      10000 non-null  float64
 1   Tot Fwd Pkts       10000 non-null  float64
 2   Tot Bwd Pkts       10000 non-null  float64
 3   TotLen Fwd Pkts    10000 non-null  float64
 4   TotLen Bwd Pkts    10000 non-null  float64
 5   Fwd Pkt Len Max    10000 non-null  float64
 6   Fwd Pkt Len Min    10000 non-null  float64
 7   Fwd Pkt Len Mean   10000 non-null  float64
 8   Fwd Pkt Len Std    10000 non-null  float64
 9   Bwd Pkt Len Max    10000 non-null  float64
 10  Bwd Pkt Len Min    10000 non-null  float64
 11  Bwd Pkt Len Mean   10000 non-null  float64
 12  Bwd Pkt Len Std    10000 non-null  float64
 13  Flow Byts/s        10000 non-null  float64
 14  Flow Pkts/s        10000 non-null  float64
 15  Flow IAT Mean      10000 non-null  float64
 16  Flow IAT Std       1000

# Load DL-based IDS model

In [8]:
model_cnn = tf.keras.models.load_model('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/insdn_cnn_bin.h5', compile=True)

# SHAP

In [9]:
explainer = shap.KernelExplainer(model_cnn, shap.sample(df, 100))

In [10]:
hsj_100 = hsj_df.head(100)

42m ~ 1g

In [11]:
shap_values = explainer.shap_values(hsj_100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
len(shap_values)

1

In [ ]:
type(shap_values[0])

numpy.ndarray

In [16]:
len(shap_values[0])

100

In [17]:
shap_values[0].shape

(100, 76)

In [13]:
data_df = pd.DataFrame(shap_values[0])

In [14]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.000031,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000003,...,0.000000,0.0,0.000000,0.000000,6.101843e-07,0.000000,0.000036,0.000009,0.000000,0.000057
1,0.000000,0.0,0.000007,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000016,0.000000,0.000000,0.000000
2,0.000000,0.0,0.000008,0.000000,0.0,0.000009,0.000000,0.000004,0.000014,0.000000,...,0.000000,0.0,0.000000,0.000026,0.000000e+00,0.000000,0.000006,0.000019,0.000000,0.000000
3,0.000000,0.0,0.000005,0.000000,0.0,0.000000,0.000004,0.000000,0.000012,0.000000,...,0.000007,0.0,0.000000,0.000000,0.000000e+00,0.000009,0.000006,0.000000,0.000000,0.000000
4,-0.000245,0.0,0.000000,-0.000036,0.0,-0.000069,0.000067,0.000000,-0.000036,0.000000,...,0.000032,0.0,0.000025,0.000045,6.224904e-05,0.000025,-0.000336,0.000000,0.000626,-0.000624


In [15]:
data_df.columns = df.columns
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      100 non-null    float64
 1   Tot Fwd Pkts       100 non-null    float64
 2   Tot Bwd Pkts       100 non-null    float64
 3   TotLen Fwd Pkts    100 non-null    float64
 4   TotLen Bwd Pkts    100 non-null    float64
 5   Fwd Pkt Len Max    100 non-null    float64
 6   Fwd Pkt Len Min    100 non-null    float64
 7   Fwd Pkt Len Mean   100 non-null    float64
 8   Fwd Pkt Len Std    100 non-null    float64
 9   Bwd Pkt Len Max    100 non-null    float64
 10  Bwd Pkt Len Min    100 non-null    float64
 11  Bwd Pkt Len Mean   100 non-null    float64
 12  Bwd Pkt Len Std    100 non-null    float64
 13  Flow Byts/s        100 non-null    float64
 14  Flow Pkts/s        100 non-null    float64
 15  Flow IAT Mean      100 non-null    float64
 16  Flow IAT Std       100 non-

In [18]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/CNN_HSJ/hsj_100_samples_[0].csv', index=False)

In [19]:
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000031,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000003,...,0.000000,0.0,0.000000,0.000000,6.101843e-07,0.000000,0.000036,0.000009,0.000000,0.000057
1,0.000000,0.0,0.000007,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000016,0.000000,0.000000,0.000000
2,0.000000,0.0,0.000008,0.000000,0.0,0.000009,0.000000,0.000004,0.000014,0.000000,...,0.000000,0.0,0.000000,0.000026,0.000000e+00,0.000000,0.000006,0.000019,0.000000,0.000000
3,0.000000,0.0,0.000005,0.000000,0.0,0.000000,0.000004,0.000000,0.000012,0.000000,...,0.000007,0.0,0.000000,0.000000,0.000000e+00,0.000009,0.000006,0.000000,0.000000,0.000000
4,-0.000245,0.0,0.000000,-0.000036,0.0,-0.000069,0.000067,0.000000,-0.000036,0.000000,...,0.000032,0.0,0.000025,0.000045,6.224904e-05,0.000025,-0.000336,0.000000,0.000626,-0.000624


In [20]:
data_dict = {}
for idx in range(0, 100):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [21]:
print(data_dict)

{'Adv sample 0': ['ACK Flag Cnt', 'SYN Flag Cnt', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd PSH Flags', 'Bwd IAT Tot', 'PSH Flag Cnt', 'URG Flag Cnt'], 'Adv sample 1': ['Bwd Pkts/s', 'FIN Flag Cnt', 'ACK Flag Cnt', 'Bwd IAT Tot', 'Init Bwd Win Byts', 'Bwd PSH Flags', 'PSH Flag Cnt', 'Flow Pkts/s', 'Down/Up Ratio', 'Flow IAT Max'], 'Adv sample 2': ['Bwd Pkts/s', 'FIN Flag Cnt', 'ACK Flag Cnt', 'Init Bwd Win Byts', 'Bwd IAT Tot', 'Flow Pkts/s', 'Bwd PSH Flags', 'Flow IAT Max', 'PSH Flag Cnt', 'Fwd IAT Tot'], 'Adv sample 3': ['Bwd Pkts/s', 'FIN Flag Cnt', 'ACK Flag Cnt', 'Init Bwd Win Byts', 'Bwd IAT Tot', 'Bwd PSH Flags', 'Flow Pkts/s', 'Flow IAT Max', 'PSH Flag Cnt', 'Fwd IAT Tot'], 'Adv sample 4': ['Down/Up Ratio', 'Idle Max', 'Bwd Pkts/s', 'Init Bwd Win Byts', 'SYN Flag Cnt', 'Bwd PSH Flags', 'Flow Pkts/s', 'Fwd IAT Tot', 'Fwd IAT Std', 'URG Flag Cnt'], 'Adv sample 5': ['ACK Flag Cnt', 'SYN Flag Cnt', 'FIN Flag Cnt', 'Init Bwd Win Byts', 'Bwd PSH Flags', '

In [22]:
len(shap_values[0])

100

In [23]:
len(data_dict)

100

# Detection phase

In [24]:
import os
import pandas as pd

In [25]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/cnn_top24_InSDN.csv')
whitelist_df.head(24)

,Feature,Frequency
0,URG Flag Cnt,149
1,Bwd Pkts/s,148
2,FIN Flag Cnt,141
3,Bwd PSH Flags,131
4,SYN Flag Cnt,123
5,PSH Flag Cnt,116
6,ACK Flag Cnt,98
7,Down/Up Ratio,97
8,Flow Pkts/s,86
9,Init Bwd Win Byts,82


## Detection rate (top 24) = 55/100

In [26]:
whitelist = whitelist_df.iloc[:, 0][:24].tolist()
print(whitelist)

['URG Flag Cnt', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Bwd PSH Flags', 'SYN Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Bwd URG Flags', 'Flow Duration', 'Idle Min', 'Bwd Pkt Len Max', 'Fwd IAT Max', 'Flow IAT Std', 'Fwd IAT Mean', 'Flow IAT Max', 'Tot Fwd Pkts', 'Bwd Pkt Len Mean', 'Bwd IAT Std', 'Idle Max']


In [27]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:55


## Detection rate (top 38) = 53/100

In [28]:
whitelist2 = whitelist_df.iloc[:, 0][:38].tolist()
print(whitelist2)

['URG Flag Cnt', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Bwd PSH Flags', 'SYN Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Bwd URG Flags', 'Flow Duration', 'Idle Min', 'Bwd Pkt Len Max', 'Fwd IAT Max', 'Flow IAT Std', 'Fwd IAT Mean', 'Flow IAT Max', 'Tot Fwd Pkts', 'Bwd Pkt Len Mean', 'Bwd IAT Std', 'Idle Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'Idle Mean', 'Bwd Seg Size Avg', 'Pkt Len Mean', 'Flow IAT Mean', 'Pkt Len Std', 'TotLen Bwd Pkts', 'Bwd Pkt Len Std', 'Bwd Header Len', 'Subflow Bwd Pkts', 'Fwd Act Data Pkts', 'Fwd Pkt Len Min', 'Pkt Len Min']


In [29]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:53
